In [1]:
from src.Block import Block
from src.ReasoningBlock import ReasoningBlock
from src.Model import Model
from src.configs import Phi2Config
from typing import List, Union
import os, json, csv
import datetime

In [2]:
def generate_time_string():
    now = datetime.datetime.now()
    month = now.month
    day = now.day
    year = now.year
    hour = now.hour
    minute = now.minute
    seconds = now.second
    time_string = f"{month}_{day}_{year}_{hour}_{minute}_{seconds}"
    return time_string

def get_queries():
    queries = []
    with open("./data/queries/batch1_2_qa.csv", "r") as f:
        reader = csv.reader(f)
        for row in reader:
            if reader.line_num == 1: continue
            queries.append(row[1])
    return queries

In [3]:
get_queries()

['What are the key factors that influence the character of the pulse?',
 'What is the recommended approach for accurately assessing the character of the pulse in the cardiovascular system?',
 'Which conditions commonly lead to a bounding pulse, characterized by high amplitude and wide pulse pressure?',
 'What is the role of aortic stenosis in ventricular emptying?',
 'What is the primary mechanism by which aortic stenosis affects the cardiovascular system?',
 'How does the venous pulse differ from the arterial pulse in terms of peaks during the cardiac cycle?',
 'What factors influence the height of the jugular venous pulse?',
 'What is a characteristic feature of tricuspid regurgitation in the cardiovascular system?',
 'Where is the normal position of the apex beat?',
 'What is the physical exam finding seen in the context of volume overload due to mitral or aortic regurgitation?',
 'What is a characteristic physical finding in cardiovascular conditions involving pressure overload, su

In [4]:
def experiment(queries: List[str], blocks: List[Union[Block, ReasoningBlock]], model: Model, exp_dir_path: str):
    log = dict()
    log['model'] = model.config.name
    
    for i, query in enumerate(queries):
        print(f"\n\n{i+1}. Query : {query}")
        
        query_dir_path = os.path.join(exp_dir_path, f"{i+1}")
        os.makedirs(query_dir_path, exist_ok=True)
        
        print("\tGenerating Outline")
        outline = model.gen_outline(query)
        
        print("\tGenerating Response")
        response = model.gen_answer(outline, query) # get original response
        log[i+1] = {'outline': outline, 'orig_res': response, "final_res": None, "points": [], "query": query}
            
        print("\tGenerating points")
        points = model.gen_points(response)
        log[i+1]['points'] = points
        
        # iterate through factuality, clarity, etc. blocks
        for j, block in enumerate(blocks):
            block_log = dict()
            
            print(f"\t{j+1}. {block.name}")
            
            print(f"\t\tGenerating Feedbacks")
            feedbacks = block.gen_feedback(query, points) # generate feedback for each point
            
            block_log['feedbacks'] = feedbacks
            
            print(f"\t\tGenerating Refined points")
            refined_points = block.gen_refined_points(points, feedbacks) # refine each point from its feedback
            
            block_log['refined_points'] = refined_points
            
            print(f"\t\tGenerating Refined Response\n")
            refined_response = model.gen_refined_response(refined_points) # generate refined response from refined points
            
            block_log['refined_response'] =  refined_response
            
            with open(os.path.join(query_dir_path, f"{block.name}.json"), 'w') as file:
                file.write(json.dumps(block_log))
            
            points = refined_points # give refined points from current block to the next block for refinement
            
        log[i+1]['final_res'] = refined_response
    
        with open(os.path.join(exp_dir_path, "query3.json"), 'w') as file:
            file.write(json.dumps(log))
        
    return log

In [5]:
model = Model(verbose=False)
data_dir_path = os.path.join("data")
misc_path = os.path.join(data_dir_path, "misc")
factual_block_path = os.path.join(data_dir_path, "factual")
plausibility_block_path = os.path.join(data_dir_path, "plausibility")
clarity_block_path = os.path.join(data_dir_path, "clarity")
reasoning_block_path = os.path.join(data_dir_path, 'reasoning')

factual_block = Block("FactualBlock", model, factual_block_path, misc_path)
plausibility_block = Block("PlausibilityBlock", model, plausibility_block_path, misc_path)
clarity_block = Block("ClarityBlock", model, clarity_block_path, misc_path)
reasoning_block = ReasoningBlock("ReasoningBlock", model, reasoning_block_path, misc_path)

queries = get_queries()

id = generate_time_string()
exp_dir_path = os.path.join("logs", id)
os.makedirs(exp_dir_path, exist_ok=True)

log = experiment(queries, [clarity_block, factual_block, plausibility_block, reasoning_block], model, exp_dir_path)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 4 CUDA devices:
  Device 0: NVIDIA RTX A6000, compute capability 8.6, VMM: yes
  Device 1: NVIDIA RTX A6000, compute capability 8.6, VMM: yes
  Device 2: NVIDIA RTX A6000, compute capability 8.6, VMM: yes
  Device 3: NVIDIA RTX A6000, compute capability 8.6, VMM: yes




1. Query : What are the key factors that influence the character of the pulse?
	Generating Response
	Generating points
	1. ClarityBlock
		Generating Feedbacks
LR||No, the point is not directly related to the query which asks about the key factors influencing the character of the pulse. Instead, it discusses potential meanings of an irregular or weak pulse, which is more indicative of a symptom rather than a factor that influences pulse character.

Feedback: Focus on the key factors that influence pulse character as per the query, not on symptoms or conditions related to an irregular or weak pulse.
F||the point is not directly related to the query which asks about the key factors influencing the character of the pulse. Instead, it discusses potential meanings of an irregular or weak pulse, which is more indicative of a symptom rather than a factor that influences pulse character.

Focus on the key factors that influence pulse character as per the query, not on symptoms or conditions r

KeyboardInterrupt: 